In [8]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()

0

In [9]:
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [10]:
#这个评估标准是大赛规定的
def rmsle(y_true, y_pred):
    return 'RMSLE', round(np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))),6), False

In [11]:
#3月1号-3月16日作为测试集，然后与3月1号的实际做对比，对3.2-3.16进行微调

In [12]:
# class lightgbm.LGBMRegressor(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, 
#                              n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, 
#                              min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
#                              subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, 
#                              random_state=None, n_jobs=-1, silent=True, importance_type='split', **kwargs)

In [64]:
#用来调最优参数的自定义函数
def get_org_lgb(data,N):
    traindf=data[data['month']!=15]
    testdf=data[data['month']==15]
    print(traindf.shape,testdf.shape)
    
    train_xy,off_xy=train_test_split(traindf,test_size=0.1,random_state=1)
    tr,val=train_test_split(train_xy,test_size=0.15,random_state=1)
    
    drop_list=['cnt','o_city_code','o_district_code','date_dt','month']
    y_train=tr.cnt
    x_train=tr.drop(drop_list,axis=1)
    
    y_val=val.cnt
    x_val=val.drop(drop_list,axis=1)
    
    y_off=off_xy.cnt
    x_off=off_xy.drop(drop_list,axis=1)
    
    test_x=testdf.drop(drop_list,axis=1)
    
    gbm = lgb.LGBMRegressor(boosting_type='gbdt',
                            objective='regression',
                            num_leaves=70,
                            max_depth=6,
                            subsample=0.8, colsample_bytree=0.8, 
                            learning_rate=0.05,
                            n_estimators=N)

    gbm.fit(x_train, y_train,
        eval_set=[(x_train,y_train),(x_val,y_val)],
        eval_metric='rmse',
        early_stopping_rounds=100)

    y_pred = gbm.predict(x_val, num_iteration=gbm.best_iteration_)
    y_eval=rmsle(y_val,y_pred)[1]
    print('The rmlse of val is:',y_eval)
#     print('Feature importances:', list(gbm.feature_importances_))
#     print('Feature importances:',(x_train.columns,gbm.feature_importances_))

    y_pred_off=gbm.predict(x_off,num_iteration=gbm.best_iteration_)
    y_off_eval=rmsle(y_off, y_pred_off)[1]
    print('The rmlse of offline is:',y_off_eval)

    gc.collect()

# =====0====

In [34]:
import time
start=time.time()
data=pd.read_csv('../data/data_0.csv')
get_org_lgb(data=data,N=942)
print("Use time is:",time.time()-start)

(273015, 39) (16980, 39)
[1]	training's rmse: 0.695223	valid_1's rmse: 0.710909
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 0.663014	valid_1's rmse: 0.677942
[3]	training's rmse: 0.632533	valid_1's rmse: 0.646718
[4]	training's rmse: 0.603698	valid_1's rmse: 0.617197
[5]	training's rmse: 0.576354	valid_1's rmse: 0.589193
[6]	training's rmse: 0.550452	valid_1's rmse: 0.562797
[7]	training's rmse: 0.525945	valid_1's rmse: 0.537715
[8]	training's rmse: 0.502893	valid_1's rmse: 0.514184
[9]	training's rmse: 0.480948	valid_1's rmse: 0.49177
[10]	training's rmse: 0.460195	valid_1's rmse: 0.470605
[11]	training's rmse: 0.440607	valid_1's rmse: 0.450626
[12]	training's rmse: 0.422062	valid_1's rmse: 0.431749
[13]	training's rmse: 0.404643	valid_1's rmse: 0.413945
[14]	training's rmse: 0.388131	valid_1's rmse: 0.397174
[15]	training's rmse: 0.372655	valid_1's rmse: 0.381382
[16]	training's rmse: 0.357954	valid_1's rmse: 0.36634
[17]	training's rmse: 0.344

[150]	training's rmse: 0.124702	valid_1's rmse: 0.133003
[151]	training's rmse: 0.124523	valid_1's rmse: 0.132898
[152]	training's rmse: 0.124438	valid_1's rmse: 0.132829
[153]	training's rmse: 0.124357	valid_1's rmse: 0.132769
[154]	training's rmse: 0.124225	valid_1's rmse: 0.132657
[155]	training's rmse: 0.124157	valid_1's rmse: 0.132594
[156]	training's rmse: 0.124089	valid_1's rmse: 0.132538
[157]	training's rmse: 0.123941	valid_1's rmse: 0.132426
[158]	training's rmse: 0.123887	valid_1's rmse: 0.132385
[159]	training's rmse: 0.123751	valid_1's rmse: 0.132272
[160]	training's rmse: 0.123646	valid_1's rmse: 0.132186
[161]	training's rmse: 0.123597	valid_1's rmse: 0.132163
[162]	training's rmse: 0.123418	valid_1's rmse: 0.131998
[163]	training's rmse: 0.123343	valid_1's rmse: 0.131942
[164]	training's rmse: 0.123203	valid_1's rmse: 0.131829
[165]	training's rmse: 0.123151	valid_1's rmse: 0.131809
[166]	training's rmse: 0.123028	valid_1's rmse: 0.131712
[167]	training's rmse: 0.122943

[300]	training's rmse: 0.114787	valid_1's rmse: 0.125534
[301]	training's rmse: 0.114759	valid_1's rmse: 0.125514
[302]	training's rmse: 0.114704	valid_1's rmse: 0.125461
[303]	training's rmse: 0.114672	valid_1's rmse: 0.125437
[304]	training's rmse: 0.114637	valid_1's rmse: 0.125423
[305]	training's rmse: 0.114606	valid_1's rmse: 0.125399
[306]	training's rmse: 0.114574	valid_1's rmse: 0.12538
[307]	training's rmse: 0.114558	valid_1's rmse: 0.125365
[308]	training's rmse: 0.114536	valid_1's rmse: 0.125348
[309]	training's rmse: 0.114461	valid_1's rmse: 0.125291
[310]	training's rmse: 0.114434	valid_1's rmse: 0.125273
[311]	training's rmse: 0.114382	valid_1's rmse: 0.125222
[312]	training's rmse: 0.114336	valid_1's rmse: 0.125187
[313]	training's rmse: 0.114304	valid_1's rmse: 0.125159
[314]	training's rmse: 0.114254	valid_1's rmse: 0.125117
[315]	training's rmse: 0.1142	valid_1's rmse: 0.125072
[316]	training's rmse: 0.114147	valid_1's rmse: 0.125034
[317]	training's rmse: 0.114099	va

[451]	training's rmse: 0.110128	valid_1's rmse: 0.122479
[452]	training's rmse: 0.110075	valid_1's rmse: 0.122444
[453]	training's rmse: 0.110043	valid_1's rmse: 0.122426
[454]	training's rmse: 0.11003	valid_1's rmse: 0.122409
[455]	training's rmse: 0.110003	valid_1's rmse: 0.122393
[456]	training's rmse: 0.109965	valid_1's rmse: 0.122368
[457]	training's rmse: 0.109959	valid_1's rmse: 0.122372
[458]	training's rmse: 0.109907	valid_1's rmse: 0.122337
[459]	training's rmse: 0.10986	valid_1's rmse: 0.122305
[460]	training's rmse: 0.109855	valid_1's rmse: 0.122302
[461]	training's rmse: 0.109816	valid_1's rmse: 0.122278
[462]	training's rmse: 0.109786	valid_1's rmse: 0.122257
[463]	training's rmse: 0.109772	valid_1's rmse: 0.12225
[464]	training's rmse: 0.109726	valid_1's rmse: 0.122212
[465]	training's rmse: 0.1097	valid_1's rmse: 0.122197
[466]	training's rmse: 0.109669	valid_1's rmse: 0.122175
[467]	training's rmse: 0.109634	valid_1's rmse: 0.122149
[468]	training's rmse: 0.109625	vali

[596]	training's rmse: 0.106852	valid_1's rmse: 0.120671
[597]	training's rmse: 0.106839	valid_1's rmse: 0.120662
[598]	training's rmse: 0.106817	valid_1's rmse: 0.120654
[599]	training's rmse: 0.106786	valid_1's rmse: 0.120643
[600]	training's rmse: 0.106767	valid_1's rmse: 0.120626
[601]	training's rmse: 0.106709	valid_1's rmse: 0.120595
[602]	training's rmse: 0.106685	valid_1's rmse: 0.120573
[603]	training's rmse: 0.106672	valid_1's rmse: 0.120568
[604]	training's rmse: 0.106647	valid_1's rmse: 0.120557
[605]	training's rmse: 0.10663	valid_1's rmse: 0.120547
[606]	training's rmse: 0.10659	valid_1's rmse: 0.120517
[607]	training's rmse: 0.106578	valid_1's rmse: 0.120512
[608]	training's rmse: 0.106568	valid_1's rmse: 0.120507
[609]	training's rmse: 0.106546	valid_1's rmse: 0.12049
[610]	training's rmse: 0.106535	valid_1's rmse: 0.120493
[611]	training's rmse: 0.106513	valid_1's rmse: 0.120485
[612]	training's rmse: 0.106488	valid_1's rmse: 0.120488
[613]	training's rmse: 0.106478	va

[745]	training's rmse: 0.104176	valid_1's rmse: 0.119611
[746]	training's rmse: 0.104148	valid_1's rmse: 0.119603
[747]	training's rmse: 0.104146	valid_1's rmse: 0.119605
[748]	training's rmse: 0.104138	valid_1's rmse: 0.119604
[749]	training's rmse: 0.104118	valid_1's rmse: 0.119589
[750]	training's rmse: 0.104098	valid_1's rmse: 0.119586
[751]	training's rmse: 0.104083	valid_1's rmse: 0.119577
[752]	training's rmse: 0.104071	valid_1's rmse: 0.119581
[753]	training's rmse: 0.10406	valid_1's rmse: 0.119571
[754]	training's rmse: 0.104055	valid_1's rmse: 0.119568
[755]	training's rmse: 0.104045	valid_1's rmse: 0.119576
[756]	training's rmse: 0.104033	valid_1's rmse: 0.119576
[757]	training's rmse: 0.104023	valid_1's rmse: 0.119575
[758]	training's rmse: 0.104005	valid_1's rmse: 0.119561
[759]	training's rmse: 0.103994	valid_1's rmse: 0.119556
[760]	training's rmse: 0.103978	valid_1's rmse: 0.119555
[761]	training's rmse: 0.103975	valid_1's rmse: 0.119556
[762]	training's rmse: 0.103968	

[895]	training's rmse: 0.102174	valid_1's rmse: 0.119077
[896]	training's rmse: 0.102152	valid_1's rmse: 0.119068
[897]	training's rmse: 0.102142	valid_1's rmse: 0.119066
[898]	training's rmse: 0.10213	valid_1's rmse: 0.119063
[899]	training's rmse: 0.102116	valid_1's rmse: 0.119055
[900]	training's rmse: 0.10209	valid_1's rmse: 0.119071
[901]	training's rmse: 0.102079	valid_1's rmse: 0.119062
[902]	training's rmse: 0.102058	valid_1's rmse: 0.119054
[903]	training's rmse: 0.102047	valid_1's rmse: 0.119049
[904]	training's rmse: 0.102039	valid_1's rmse: 0.119044
[905]	training's rmse: 0.102023	valid_1's rmse: 0.119043
[906]	training's rmse: 0.102022	valid_1's rmse: 0.119042
[907]	training's rmse: 0.102008	valid_1's rmse: 0.119043
[908]	training's rmse: 0.101985	valid_1's rmse: 0.119036
[909]	training's rmse: 0.10198	valid_1's rmse: 0.119032
[910]	training's rmse: 0.101967	valid_1's rmse: 0.11903
[911]	training's rmse: 0.10196	valid_1's rmse: 0.119024
[912]	training's rmse: 0.101953	vali

In [ ]:
# num_leaves=70,max_depth=6，N=942,subsample=0.8, colsample_bytree=0.8,learning_rate=0.05
# The rmlse of prediction is: 0.05626
# The rmlse of prediction is: 0.054687

In [53]:
def get_lgb(data,i,N):
    traindf=data[data['month']!=15]
    testdf=data[data['month']==15]
    print(traindf.shape,testdf.shape)
    
    train_xy,off_xy=train_test_split(traindf,test_size=0.1,random_state=1)
    tr,val=train_test_split(train_xy,test_size=0.15,random_state=1)
    
    drop_list=['cnt','o_city_code','o_district_code','date_dt','month']
    
    #用于后面的数据融合
    y_traindf=traindf.cnt
    x_traindf=traindf.drop(drop_list,axis=1)
    
    #将traindf分成训练和线下验证二种，训练集又分成训练和验证二种
    y_train=tr.cnt
    x_train=tr.drop(drop_list,axis=1)
    
    y_val=val.cnt
    x_val=val.drop(drop_list,axis=1)
    
    y_off=off_xy.cnt
    x_off=off_xy.drop(drop_list,axis=1)
    
    test_x=testdf.drop(drop_list,axis=1)
    
    gbm = lgb.LGBMRegressor(boosting_type='gbdt',
                            objective='regression',
                            num_leaves=70,
                            max_depth=6,
                            subsample=0.8, colsample_bytree=0.8, 
                            learning_rate=0.05,
                            n_estimators=N)

    gbm.fit(x_train, y_train,
        eval_set=[(x_train,y_train),(x_val,y_val)],
        eval_metric='rmse',
        early_stopping_rounds=100)

#     y_pred = gbm.predict(x_val, num_iteration=gbm.best_iteration_)
#     y_eval=rmsle(y_val,y_pred)[1]
#     print('The rmlse of val is:',y_eval)

#     y_pred_off=gbm.predict(x_off,num_iteration=gbm.best_iteration_)
#     y_off_eval=rmsle(y_off, y_pred_off)[1]
#     print('The rmlse of offline is:',y_off_eval)

    #预测
    traindf['cnt_p']=gbm.predict(x_traindf,num_iteration=gbm.best_iteration_)
    testdf['cnt_p']=gbm.predict(test_x,num_iteration=gbm.best_iteration_)
    
    #保存
    tr=traindf[['date_dt','start_address','end_address','cnt','cnt_p']]
    tt=testdf[['date_dt','start_address','end_address','cnt','cnt_p']]
    tr.to_csv('..\data\data_train_'+str(i)+'.csv',index=False)
    tt.to_csv('..\data\data_test_'+str(i)+'.csv',index=False) 
    
    gc.collect()

In [38]:
start=time.time()
get_lgb(data=data,i=0,N=942)
print("Use time is:",time.time()-start)

(273015, 39) (16980, 39)
[1]	training's rmse: 0.695223	valid_1's rmse: 0.710909
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 0.663014	valid_1's rmse: 0.677942
[3]	training's rmse: 0.632533	valid_1's rmse: 0.646718
[4]	training's rmse: 0.603698	valid_1's rmse: 0.617197
[5]	training's rmse: 0.576354	valid_1's rmse: 0.589193
[6]	training's rmse: 0.550452	valid_1's rmse: 0.562797
[7]	training's rmse: 0.525945	valid_1's rmse: 0.537715
[8]	training's rmse: 0.502893	valid_1's rmse: 0.514184
[9]	training's rmse: 0.480948	valid_1's rmse: 0.49177
[10]	training's rmse: 0.460195	valid_1's rmse: 0.470605
[11]	training's rmse: 0.440607	valid_1's rmse: 0.450626
[12]	training's rmse: 0.422062	valid_1's rmse: 0.431749
[13]	training's rmse: 0.404643	valid_1's rmse: 0.413945
[14]	training's rmse: 0.388131	valid_1's rmse: 0.397174
[15]	training's rmse: 0.372655	valid_1's rmse: 0.381382
[16]	training's rmse: 0.357954	valid_1's rmse: 0.36634
[17]	training's rmse: 0.344

[152]	training's rmse: 0.124438	valid_1's rmse: 0.132829
[153]	training's rmse: 0.124357	valid_1's rmse: 0.132769
[154]	training's rmse: 0.124225	valid_1's rmse: 0.132657
[155]	training's rmse: 0.124157	valid_1's rmse: 0.132594
[156]	training's rmse: 0.124089	valid_1's rmse: 0.132538
[157]	training's rmse: 0.123941	valid_1's rmse: 0.132426
[158]	training's rmse: 0.123887	valid_1's rmse: 0.132385
[159]	training's rmse: 0.123751	valid_1's rmse: 0.132272
[160]	training's rmse: 0.123646	valid_1's rmse: 0.132186
[161]	training's rmse: 0.123597	valid_1's rmse: 0.132163
[162]	training's rmse: 0.123418	valid_1's rmse: 0.131998
[163]	training's rmse: 0.123343	valid_1's rmse: 0.131942
[164]	training's rmse: 0.123203	valid_1's rmse: 0.131829
[165]	training's rmse: 0.123151	valid_1's rmse: 0.131809
[166]	training's rmse: 0.123028	valid_1's rmse: 0.131712
[167]	training's rmse: 0.122943	valid_1's rmse: 0.13165
[168]	training's rmse: 0.122885	valid_1's rmse: 0.131603
[169]	training's rmse: 0.122737	

[299]	training's rmse: 0.114827	valid_1's rmse: 0.125573
[300]	training's rmse: 0.114787	valid_1's rmse: 0.125534
[301]	training's rmse: 0.114759	valid_1's rmse: 0.125514
[302]	training's rmse: 0.114704	valid_1's rmse: 0.125461
[303]	training's rmse: 0.114672	valid_1's rmse: 0.125437
[304]	training's rmse: 0.114637	valid_1's rmse: 0.125423
[305]	training's rmse: 0.114606	valid_1's rmse: 0.125399
[306]	training's rmse: 0.114574	valid_1's rmse: 0.12538
[307]	training's rmse: 0.114558	valid_1's rmse: 0.125365
[308]	training's rmse: 0.114536	valid_1's rmse: 0.125348
[309]	training's rmse: 0.114461	valid_1's rmse: 0.125291
[310]	training's rmse: 0.114434	valid_1's rmse: 0.125273
[311]	training's rmse: 0.114382	valid_1's rmse: 0.125222
[312]	training's rmse: 0.114336	valid_1's rmse: 0.125187
[313]	training's rmse: 0.114304	valid_1's rmse: 0.125159
[314]	training's rmse: 0.114254	valid_1's rmse: 0.125117
[315]	training's rmse: 0.1142	valid_1's rmse: 0.125072
[316]	training's rmse: 0.114147	va

[449]	training's rmse: 0.110188	valid_1's rmse: 0.122509
[450]	training's rmse: 0.11016	valid_1's rmse: 0.122508
[451]	training's rmse: 0.110128	valid_1's rmse: 0.122479
[452]	training's rmse: 0.110075	valid_1's rmse: 0.122444
[453]	training's rmse: 0.110043	valid_1's rmse: 0.122426
[454]	training's rmse: 0.11003	valid_1's rmse: 0.122409
[455]	training's rmse: 0.110003	valid_1's rmse: 0.122393
[456]	training's rmse: 0.109965	valid_1's rmse: 0.122368
[457]	training's rmse: 0.109959	valid_1's rmse: 0.122372
[458]	training's rmse: 0.109907	valid_1's rmse: 0.122337
[459]	training's rmse: 0.10986	valid_1's rmse: 0.122305
[460]	training's rmse: 0.109855	valid_1's rmse: 0.122302
[461]	training's rmse: 0.109816	valid_1's rmse: 0.122278
[462]	training's rmse: 0.109786	valid_1's rmse: 0.122257
[463]	training's rmse: 0.109772	valid_1's rmse: 0.12225
[464]	training's rmse: 0.109726	valid_1's rmse: 0.122212
[465]	training's rmse: 0.1097	valid_1's rmse: 0.122197
[466]	training's rmse: 0.109669	valid

[595]	training's rmse: 0.106873	valid_1's rmse: 0.120692
[596]	training's rmse: 0.106852	valid_1's rmse: 0.120671
[597]	training's rmse: 0.106839	valid_1's rmse: 0.120662
[598]	training's rmse: 0.106817	valid_1's rmse: 0.120654
[599]	training's rmse: 0.106786	valid_1's rmse: 0.120643
[600]	training's rmse: 0.106767	valid_1's rmse: 0.120626
[601]	training's rmse: 0.106709	valid_1's rmse: 0.120595
[602]	training's rmse: 0.106685	valid_1's rmse: 0.120573
[603]	training's rmse: 0.106672	valid_1's rmse: 0.120568
[604]	training's rmse: 0.106647	valid_1's rmse: 0.120557
[605]	training's rmse: 0.10663	valid_1's rmse: 0.120547
[606]	training's rmse: 0.10659	valid_1's rmse: 0.120517
[607]	training's rmse: 0.106578	valid_1's rmse: 0.120512
[608]	training's rmse: 0.106568	valid_1's rmse: 0.120507
[609]	training's rmse: 0.106546	valid_1's rmse: 0.12049
[610]	training's rmse: 0.106535	valid_1's rmse: 0.120493
[611]	training's rmse: 0.106513	valid_1's rmse: 0.120485
[612]	training's rmse: 0.106488	va

[745]	training's rmse: 0.104176	valid_1's rmse: 0.119611
[746]	training's rmse: 0.104148	valid_1's rmse: 0.119603
[747]	training's rmse: 0.104146	valid_1's rmse: 0.119605
[748]	training's rmse: 0.104138	valid_1's rmse: 0.119604
[749]	training's rmse: 0.104118	valid_1's rmse: 0.119589
[750]	training's rmse: 0.104098	valid_1's rmse: 0.119586
[751]	training's rmse: 0.104083	valid_1's rmse: 0.119577
[752]	training's rmse: 0.104071	valid_1's rmse: 0.119581
[753]	training's rmse: 0.10406	valid_1's rmse: 0.119571
[754]	training's rmse: 0.104055	valid_1's rmse: 0.119568
[755]	training's rmse: 0.104045	valid_1's rmse: 0.119576
[756]	training's rmse: 0.104033	valid_1's rmse: 0.119576
[757]	training's rmse: 0.104023	valid_1's rmse: 0.119575
[758]	training's rmse: 0.104005	valid_1's rmse: 0.119561
[759]	training's rmse: 0.103994	valid_1's rmse: 0.119556
[760]	training's rmse: 0.103978	valid_1's rmse: 0.119555
[761]	training's rmse: 0.103975	valid_1's rmse: 0.119556
[762]	training's rmse: 0.103968	

[897]	training's rmse: 0.102142	valid_1's rmse: 0.119066
[898]	training's rmse: 0.10213	valid_1's rmse: 0.119063
[899]	training's rmse: 0.102116	valid_1's rmse: 0.119055
[900]	training's rmse: 0.10209	valid_1's rmse: 0.119071
[901]	training's rmse: 0.102079	valid_1's rmse: 0.119062
[902]	training's rmse: 0.102058	valid_1's rmse: 0.119054
[903]	training's rmse: 0.102047	valid_1's rmse: 0.119049
[904]	training's rmse: 0.102039	valid_1's rmse: 0.119044
[905]	training's rmse: 0.102023	valid_1's rmse: 0.119043
[906]	training's rmse: 0.102022	valid_1's rmse: 0.119042
[907]	training's rmse: 0.102008	valid_1's rmse: 0.119043
[908]	training's rmse: 0.101985	valid_1's rmse: 0.119036
[909]	training's rmse: 0.10198	valid_1's rmse: 0.119032
[910]	training's rmse: 0.101967	valid_1's rmse: 0.11903
[911]	training's rmse: 0.10196	valid_1's rmse: 0.119024
[912]	training's rmse: 0.101953	valid_1's rmse: 0.11902
[913]	training's rmse: 0.101926	valid_1's rmse: 0.119028
[914]	training's rmse: 0.101915	valid

#  ==============0 over==============

In [52]:
#2开始
start=time.time()
data2=pd.read_csv('../data/data_2.csv')
get_org_lgb(data=data2,N=1000)
print("Use time is:",time.time()-start)

(256286, 39) (15500, 39)
[1]	training's rmse: 1.30345	valid_1's rmse: 1.29396
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 1.24251	valid_1's rmse: 1.23331
[3]	training's rmse: 1.18478	valid_1's rmse: 1.17578
[4]	training's rmse: 1.13016	valid_1's rmse: 1.12144
[5]	training's rmse: 1.07842	valid_1's rmse: 1.07002
[6]	training's rmse: 1.0294	valid_1's rmse: 1.02139
[7]	training's rmse: 0.983051	valid_1's rmse: 0.975142
[8]	training's rmse: 0.939348	valid_1's rmse: 0.931546
[9]	training's rmse: 0.897849	valid_1's rmse: 0.890388
[10]	training's rmse: 0.858489	valid_1's rmse: 0.851312
[11]	training's rmse: 0.821313	valid_1's rmse: 0.814407
[12]	training's rmse: 0.786225	valid_1's rmse: 0.77948
[13]	training's rmse: 0.753223	valid_1's rmse: 0.74683
[14]	training's rmse: 0.721906	valid_1's rmse: 0.715741
[15]	training's rmse: 0.692564	valid_1's rmse: 0.686702
[16]	training's rmse: 0.664734	valid_1's rmse: 0.659119
[17]	training's rmse: 0.638749	valid_1's

[152]	training's rmse: 0.252943	valid_1's rmse: 0.264262
[153]	training's rmse: 0.252717	valid_1's rmse: 0.264089
[154]	training's rmse: 0.252476	valid_1's rmse: 0.263869
[155]	training's rmse: 0.252349	valid_1's rmse: 0.263752
[156]	training's rmse: 0.252213	valid_1's rmse: 0.263635
[157]	training's rmse: 0.251935	valid_1's rmse: 0.263386
[158]	training's rmse: 0.251787	valid_1's rmse: 0.263267
[159]	training's rmse: 0.251565	valid_1's rmse: 0.263075
[160]	training's rmse: 0.251446	valid_1's rmse: 0.262964
[161]	training's rmse: 0.251276	valid_1's rmse: 0.262811
[162]	training's rmse: 0.251076	valid_1's rmse: 0.262647
[163]	training's rmse: 0.251004	valid_1's rmse: 0.262582
[164]	training's rmse: 0.250628	valid_1's rmse: 0.262236
[165]	training's rmse: 0.250438	valid_1's rmse: 0.262055
[166]	training's rmse: 0.250312	valid_1's rmse: 0.261954
[167]	training's rmse: 0.250222	valid_1's rmse: 0.261882
[168]	training's rmse: 0.250076	valid_1's rmse: 0.261773
[169]	training's rmse: 0.249892

[304]	training's rmse: 0.237198	valid_1's rmse: 0.251366
[305]	training's rmse: 0.237169	valid_1's rmse: 0.251348
[306]	training's rmse: 0.237143	valid_1's rmse: 0.251328
[307]	training's rmse: 0.237089	valid_1's rmse: 0.25128
[308]	training's rmse: 0.237063	valid_1's rmse: 0.251259
[309]	training's rmse: 0.236979	valid_1's rmse: 0.251206
[310]	training's rmse: 0.236963	valid_1's rmse: 0.251198
[311]	training's rmse: 0.236844	valid_1's rmse: 0.251135
[312]	training's rmse: 0.236817	valid_1's rmse: 0.251127
[313]	training's rmse: 0.236801	valid_1's rmse: 0.251118
[314]	training's rmse: 0.236782	valid_1's rmse: 0.251103
[315]	training's rmse: 0.236749	valid_1's rmse: 0.251083
[316]	training's rmse: 0.23665	valid_1's rmse: 0.251008
[317]	training's rmse: 0.23657	valid_1's rmse: 0.25095
[318]	training's rmse: 0.236458	valid_1's rmse: 0.250857
[319]	training's rmse: 0.236435	valid_1's rmse: 0.250843
[320]	training's rmse: 0.236372	valid_1's rmse: 0.250798
[321]	training's rmse: 0.236304	val

[452]	training's rmse: 0.229317	valid_1's rmse: 0.246083
[453]	training's rmse: 0.22926	valid_1's rmse: 0.246051
[454]	training's rmse: 0.229239	valid_1's rmse: 0.246036
[455]	training's rmse: 0.229179	valid_1's rmse: 0.245991
[456]	training's rmse: 0.229155	valid_1's rmse: 0.245981
[457]	training's rmse: 0.229133	valid_1's rmse: 0.245965
[458]	training's rmse: 0.229057	valid_1's rmse: 0.245909
[459]	training's rmse: 0.228996	valid_1's rmse: 0.245857
[460]	training's rmse: 0.228988	valid_1's rmse: 0.245853
[461]	training's rmse: 0.228971	valid_1's rmse: 0.245846
[462]	training's rmse: 0.228926	valid_1's rmse: 0.245812
[463]	training's rmse: 0.228905	valid_1's rmse: 0.245819
[464]	training's rmse: 0.228886	valid_1's rmse: 0.245797
[465]	training's rmse: 0.228858	valid_1's rmse: 0.245777
[466]	training's rmse: 0.228829	valid_1's rmse: 0.245763
[467]	training's rmse: 0.228815	valid_1's rmse: 0.245755
[468]	training's rmse: 0.228795	valid_1's rmse: 0.245748
[469]	training's rmse: 0.228786	

[604]	training's rmse: 0.224142	valid_1's rmse: 0.243466
[605]	training's rmse: 0.224098	valid_1's rmse: 0.243463
[606]	training's rmse: 0.224052	valid_1's rmse: 0.243436
[607]	training's rmse: 0.22404	valid_1's rmse: 0.243434
[608]	training's rmse: 0.224001	valid_1's rmse: 0.243437
[609]	training's rmse: 0.223971	valid_1's rmse: 0.243441
[610]	training's rmse: 0.223927	valid_1's rmse: 0.243418
[611]	training's rmse: 0.223885	valid_1's rmse: 0.24338
[612]	training's rmse: 0.223847	valid_1's rmse: 0.243374
[613]	training's rmse: 0.223832	valid_1's rmse: 0.243376
[614]	training's rmse: 0.223805	valid_1's rmse: 0.24336
[615]	training's rmse: 0.223797	valid_1's rmse: 0.243353
[616]	training's rmse: 0.223766	valid_1's rmse: 0.243341
[617]	training's rmse: 0.223722	valid_1's rmse: 0.243336
[618]	training's rmse: 0.223675	valid_1's rmse: 0.243317
[619]	training's rmse: 0.223666	valid_1's rmse: 0.243321
[620]	training's rmse: 0.223642	valid_1's rmse: 0.243319
[621]	training's rmse: 0.223615	va

[756]	training's rmse: 0.219826	valid_1's rmse: 0.242113
[757]	training's rmse: 0.219802	valid_1's rmse: 0.242102
[758]	training's rmse: 0.219787	valid_1's rmse: 0.242093
[759]	training's rmse: 0.219776	valid_1's rmse: 0.242087
[760]	training's rmse: 0.219761	valid_1's rmse: 0.242097
[761]	training's rmse: 0.219736	valid_1's rmse: 0.242097
[762]	training's rmse: 0.219699	valid_1's rmse: 0.242098
[763]	training's rmse: 0.219688	valid_1's rmse: 0.242094
[764]	training's rmse: 0.219669	valid_1's rmse: 0.242096
[765]	training's rmse: 0.21965	valid_1's rmse: 0.242096
[766]	training's rmse: 0.219625	valid_1's rmse: 0.242091
[767]	training's rmse: 0.21959	valid_1's rmse: 0.242084
[768]	training's rmse: 0.219576	valid_1's rmse: 0.242097
[769]	training's rmse: 0.219526	valid_1's rmse: 0.242068
[770]	training's rmse: 0.219504	valid_1's rmse: 0.242056
[771]	training's rmse: 0.219458	valid_1's rmse: 0.242044
[772]	training's rmse: 0.219448	valid_1's rmse: 0.242048
[773]	training's rmse: 0.219424	v

[905]	training's rmse: 0.216651	valid_1's rmse: 0.241314
[906]	training's rmse: 0.216642	valid_1's rmse: 0.24131
[907]	training's rmse: 0.216627	valid_1's rmse: 0.241302
[908]	training's rmse: 0.216592	valid_1's rmse: 0.241299
[909]	training's rmse: 0.216588	valid_1's rmse: 0.241298
[910]	training's rmse: 0.216542	valid_1's rmse: 0.241308
[911]	training's rmse: 0.216517	valid_1's rmse: 0.241334
[912]	training's rmse: 0.216507	valid_1's rmse: 0.241335
[913]	training's rmse: 0.21649	valid_1's rmse: 0.241328
[914]	training's rmse: 0.216487	valid_1's rmse: 0.241327
[915]	training's rmse: 0.216475	valid_1's rmse: 0.241325
[916]	training's rmse: 0.216456	valid_1's rmse: 0.241311
[917]	training's rmse: 0.21644	valid_1's rmse: 0.241306
[918]	training's rmse: 0.216435	valid_1's rmse: 0.241304
[919]	training's rmse: 0.216403	valid_1's rmse: 0.24129
[920]	training's rmse: 0.216394	valid_1's rmse: 0.241284
[921]	training's rmse: 0.216382	valid_1's rmse: 0.241283
[922]	training's rmse: 0.216356	val

In [54]:
# The rmlse of val is: 0.107022
# The rmlse of offline is: 0.108167
get_lgb(data=data2,i=2,N=1000)

(256286, 39) (15500, 39)
[1]	training's rmse: 1.30345	valid_1's rmse: 1.29396
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 1.24251	valid_1's rmse: 1.23331
[3]	training's rmse: 1.18478	valid_1's rmse: 1.17578
[4]	training's rmse: 1.13016	valid_1's rmse: 1.12144
[5]	training's rmse: 1.07842	valid_1's rmse: 1.07002
[6]	training's rmse: 1.0294	valid_1's rmse: 1.02139
[7]	training's rmse: 0.983051	valid_1's rmse: 0.975142
[8]	training's rmse: 0.939348	valid_1's rmse: 0.931546
[9]	training's rmse: 0.897849	valid_1's rmse: 0.890388
[10]	training's rmse: 0.858489	valid_1's rmse: 0.851312
[11]	training's rmse: 0.821313	valid_1's rmse: 0.814407
[12]	training's rmse: 0.786225	valid_1's rmse: 0.77948
[13]	training's rmse: 0.753223	valid_1's rmse: 0.74683
[14]	training's rmse: 0.721906	valid_1's rmse: 0.715741
[15]	training's rmse: 0.692564	valid_1's rmse: 0.686702
[16]	training's rmse: 0.664734	valid_1's rmse: 0.659119
[17]	training's rmse: 0.638749	valid_1's

[146]	training's rmse: 0.254111	valid_1's rmse: 0.265281
[147]	training's rmse: 0.254054	valid_1's rmse: 0.265232
[148]	training's rmse: 0.253904	valid_1's rmse: 0.265129
[149]	training's rmse: 0.253534	valid_1's rmse: 0.264802
[150]	training's rmse: 0.253266	valid_1's rmse: 0.264558
[151]	training's rmse: 0.253136	valid_1's rmse: 0.264434
[152]	training's rmse: 0.252943	valid_1's rmse: 0.264262
[153]	training's rmse: 0.252717	valid_1's rmse: 0.264089
[154]	training's rmse: 0.252476	valid_1's rmse: 0.263869
[155]	training's rmse: 0.252349	valid_1's rmse: 0.263752
[156]	training's rmse: 0.252213	valid_1's rmse: 0.263635
[157]	training's rmse: 0.251935	valid_1's rmse: 0.263386
[158]	training's rmse: 0.251787	valid_1's rmse: 0.263267
[159]	training's rmse: 0.251565	valid_1's rmse: 0.263075
[160]	training's rmse: 0.251446	valid_1's rmse: 0.262964
[161]	training's rmse: 0.251276	valid_1's rmse: 0.262811
[162]	training's rmse: 0.251076	valid_1's rmse: 0.262647
[163]	training's rmse: 0.251004

[296]	training's rmse: 0.237628	valid_1's rmse: 0.251578
[297]	training's rmse: 0.237552	valid_1's rmse: 0.251562
[298]	training's rmse: 0.237532	valid_1's rmse: 0.251553
[299]	training's rmse: 0.23747	valid_1's rmse: 0.251506
[300]	training's rmse: 0.23742	valid_1's rmse: 0.251476
[301]	training's rmse: 0.23739	valid_1's rmse: 0.251454
[302]	training's rmse: 0.237365	valid_1's rmse: 0.251433
[303]	training's rmse: 0.237299	valid_1's rmse: 0.25141
[304]	training's rmse: 0.237198	valid_1's rmse: 0.251366
[305]	training's rmse: 0.237169	valid_1's rmse: 0.251348
[306]	training's rmse: 0.237143	valid_1's rmse: 0.251328
[307]	training's rmse: 0.237089	valid_1's rmse: 0.25128
[308]	training's rmse: 0.237063	valid_1's rmse: 0.251259
[309]	training's rmse: 0.236979	valid_1's rmse: 0.251206
[310]	training's rmse: 0.236963	valid_1's rmse: 0.251198
[311]	training's rmse: 0.236844	valid_1's rmse: 0.251135
[312]	training's rmse: 0.236817	valid_1's rmse: 0.251127
[313]	training's rmse: 0.236801	vali

[441]	training's rmse: 0.229752	valid_1's rmse: 0.246263
[442]	training's rmse: 0.229705	valid_1's rmse: 0.246257
[443]	training's rmse: 0.22967	valid_1's rmse: 0.24625
[444]	training's rmse: 0.229617	valid_1's rmse: 0.246253
[445]	training's rmse: 0.229569	valid_1's rmse: 0.246221
[446]	training's rmse: 0.229517	valid_1's rmse: 0.246201
[447]	training's rmse: 0.229508	valid_1's rmse: 0.246197
[448]	training's rmse: 0.229486	valid_1's rmse: 0.24619
[449]	training's rmse: 0.229445	valid_1's rmse: 0.246156
[450]	training's rmse: 0.229415	valid_1's rmse: 0.24613
[451]	training's rmse: 0.229378	valid_1's rmse: 0.246104
[452]	training's rmse: 0.229317	valid_1's rmse: 0.246083
[453]	training's rmse: 0.22926	valid_1's rmse: 0.246051
[454]	training's rmse: 0.229239	valid_1's rmse: 0.246036
[455]	training's rmse: 0.229179	valid_1's rmse: 0.245991
[456]	training's rmse: 0.229155	valid_1's rmse: 0.245981
[457]	training's rmse: 0.229133	valid_1's rmse: 0.245965
[458]	training's rmse: 0.229057	vali

[591]	training's rmse: 0.224537	valid_1's rmse: 0.243597
[592]	training's rmse: 0.2245	valid_1's rmse: 0.243592
[593]	training's rmse: 0.224462	valid_1's rmse: 0.243592
[594]	training's rmse: 0.224457	valid_1's rmse: 0.243589
[595]	training's rmse: 0.224426	valid_1's rmse: 0.243564
[596]	training's rmse: 0.224403	valid_1's rmse: 0.243564
[597]	training's rmse: 0.224349	valid_1's rmse: 0.243546
[598]	training's rmse: 0.224332	valid_1's rmse: 0.243532
[599]	training's rmse: 0.22431	valid_1's rmse: 0.243536
[600]	training's rmse: 0.224263	valid_1's rmse: 0.243514
[601]	training's rmse: 0.224225	valid_1's rmse: 0.243508
[602]	training's rmse: 0.224205	valid_1's rmse: 0.243494
[603]	training's rmse: 0.224174	valid_1's rmse: 0.24349
[604]	training's rmse: 0.224142	valid_1's rmse: 0.243466
[605]	training's rmse: 0.224098	valid_1's rmse: 0.243463
[606]	training's rmse: 0.224052	valid_1's rmse: 0.243436
[607]	training's rmse: 0.22404	valid_1's rmse: 0.243434
[608]	training's rmse: 0.224001	vali

[737]	training's rmse: 0.220381	valid_1's rmse: 0.242299
[738]	training's rmse: 0.220353	valid_1's rmse: 0.242268
[739]	training's rmse: 0.220318	valid_1's rmse: 0.242247
[740]	training's rmse: 0.220231	valid_1's rmse: 0.242188
[741]	training's rmse: 0.220217	valid_1's rmse: 0.242184
[742]	training's rmse: 0.22021	valid_1's rmse: 0.242182
[743]	training's rmse: 0.220185	valid_1's rmse: 0.242174
[744]	training's rmse: 0.22016	valid_1's rmse: 0.242167
[745]	training's rmse: 0.220131	valid_1's rmse: 0.24219
[746]	training's rmse: 0.22011	valid_1's rmse: 0.242183
[747]	training's rmse: 0.220061	valid_1's rmse: 0.242181
[748]	training's rmse: 0.219987	valid_1's rmse: 0.242132
[749]	training's rmse: 0.219976	valid_1's rmse: 0.242144
[750]	training's rmse: 0.219959	valid_1's rmse: 0.242138
[751]	training's rmse: 0.219946	valid_1's rmse: 0.242137
[752]	training's rmse: 0.219934	valid_1's rmse: 0.242129
[753]	training's rmse: 0.219888	valid_1's rmse: 0.242133
[754]	training's rmse: 0.219857	val

[884]	training's rmse: 0.217073	valid_1's rmse: 0.241408
[885]	training's rmse: 0.217047	valid_1's rmse: 0.241436
[886]	training's rmse: 0.217036	valid_1's rmse: 0.241437
[887]	training's rmse: 0.217	valid_1's rmse: 0.241414
[888]	training's rmse: 0.21699	valid_1's rmse: 0.241405
[889]	training's rmse: 0.216961	valid_1's rmse: 0.24139
[890]	training's rmse: 0.216936	valid_1's rmse: 0.24139
[891]	training's rmse: 0.21691	valid_1's rmse: 0.241385
[892]	training's rmse: 0.21689	valid_1's rmse: 0.241377
[893]	training's rmse: 0.21688	valid_1's rmse: 0.241367
[894]	training's rmse: 0.216856	valid_1's rmse: 0.241391
[895]	training's rmse: 0.21684	valid_1's rmse: 0.241389
[896]	training's rmse: 0.216821	valid_1's rmse: 0.241366
[897]	training's rmse: 0.216795	valid_1's rmse: 0.241347
[898]	training's rmse: 0.216758	valid_1's rmse: 0.241334
[899]	training's rmse: 0.216736	valid_1's rmse: 0.241329
[900]	training's rmse: 0.216722	valid_1's rmse: 0.241333
[901]	training's rmse: 0.216713	valid_1's

# =========2 over==========

In [57]:
# 1
data1=pd.read_csv('../data/data_1.csv')
get_org_lgb(data=data1,N=2500)

(567719, 39) (34104, 39)
[1]	training's rmse: 0.943659	valid_1's rmse: 0.956605
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 0.898686	valid_1's rmse: 0.910827
[3]	training's rmse: 0.856048	valid_1's rmse: 0.867616
[4]	training's rmse: 0.815653	valid_1's rmse: 0.826457
[5]	training's rmse: 0.777377	valid_1's rmse: 0.787464
[6]	training's rmse: 0.741162	valid_1's rmse: 0.750723
[7]	training's rmse: 0.706819	valid_1's rmse: 0.715779
[8]	training's rmse: 0.674389	valid_1's rmse: 0.682824
[9]	training's rmse: 0.643591	valid_1's rmse: 0.651472
[10]	training's rmse: 0.614412	valid_1's rmse: 0.621825
[11]	training's rmse: 0.586844	valid_1's rmse: 0.593864
[12]	training's rmse: 0.560744	valid_1's rmse: 0.567435
[13]	training's rmse: 0.53613	valid_1's rmse: 0.54239
[14]	training's rmse: 0.512832	valid_1's rmse: 0.518754
[15]	training's rmse: 0.490886	valid_1's rmse: 0.496474
[16]	training's rmse: 0.470157	valid_1's rmse: 0.475478
[17]	training's rmse: 0.450

[147]	training's rmse: 0.162185	valid_1's rmse: 0.167093
[148]	training's rmse: 0.1621	valid_1's rmse: 0.167009
[149]	training's rmse: 0.161955	valid_1's rmse: 0.16688
[150]	training's rmse: 0.161819	valid_1's rmse: 0.166761
[151]	training's rmse: 0.161678	valid_1's rmse: 0.166627
[152]	training's rmse: 0.161548	valid_1's rmse: 0.166501
[153]	training's rmse: 0.161405	valid_1's rmse: 0.166359
[154]	training's rmse: 0.16123	valid_1's rmse: 0.16618
[155]	training's rmse: 0.161092	valid_1's rmse: 0.16604
[156]	training's rmse: 0.160945	valid_1's rmse: 0.165896
[157]	training's rmse: 0.160828	valid_1's rmse: 0.165779
[158]	training's rmse: 0.160771	valid_1's rmse: 0.165726
[159]	training's rmse: 0.160691	valid_1's rmse: 0.165654
[160]	training's rmse: 0.160586	valid_1's rmse: 0.165547
[161]	training's rmse: 0.160489	valid_1's rmse: 0.165471
[162]	training's rmse: 0.160396	valid_1's rmse: 0.165388
[163]	training's rmse: 0.160316	valid_1's rmse: 0.165314
[164]	training's rmse: 0.160229	valid

[294]	training's rmse: 0.152549	valid_1's rmse: 0.158681
[295]	training's rmse: 0.152517	valid_1's rmse: 0.158652
[296]	training's rmse: 0.152465	valid_1's rmse: 0.158611
[297]	training's rmse: 0.15244	valid_1's rmse: 0.158593
[298]	training's rmse: 0.1524	valid_1's rmse: 0.15856
[299]	training's rmse: 0.152383	valid_1's rmse: 0.158549
[300]	training's rmse: 0.152356	valid_1's rmse: 0.158534
[301]	training's rmse: 0.152313	valid_1's rmse: 0.158504
[302]	training's rmse: 0.152279	valid_1's rmse: 0.158474
[303]	training's rmse: 0.152259	valid_1's rmse: 0.158457
[304]	training's rmse: 0.152211	valid_1's rmse: 0.15842
[305]	training's rmse: 0.152185	valid_1's rmse: 0.158399
[306]	training's rmse: 0.152161	valid_1's rmse: 0.158376
[307]	training's rmse: 0.152124	valid_1's rmse: 0.158349
[308]	training's rmse: 0.152099	valid_1's rmse: 0.158336
[309]	training's rmse: 0.152074	valid_1's rmse: 0.158323
[310]	training's rmse: 0.152045	valid_1's rmse: 0.158306
[311]	training's rmse: 0.152017	vali

[439]	training's rmse: 0.148455	valid_1's rmse: 0.155651
[440]	training's rmse: 0.148431	valid_1's rmse: 0.155636
[441]	training's rmse: 0.148405	valid_1's rmse: 0.155625
[442]	training's rmse: 0.148388	valid_1's rmse: 0.155617
[443]	training's rmse: 0.148362	valid_1's rmse: 0.1556
[444]	training's rmse: 0.148346	valid_1's rmse: 0.155592
[445]	training's rmse: 0.14832	valid_1's rmse: 0.155575
[446]	training's rmse: 0.148296	valid_1's rmse: 0.15557
[447]	training's rmse: 0.148262	valid_1's rmse: 0.155548
[448]	training's rmse: 0.148245	valid_1's rmse: 0.155535
[449]	training's rmse: 0.148222	valid_1's rmse: 0.155516
[450]	training's rmse: 0.148201	valid_1's rmse: 0.155502
[451]	training's rmse: 0.148165	valid_1's rmse: 0.155469
[452]	training's rmse: 0.148154	valid_1's rmse: 0.155464
[453]	training's rmse: 0.148145	valid_1's rmse: 0.155459
[454]	training's rmse: 0.148128	valid_1's rmse: 0.155448
[455]	training's rmse: 0.148118	valid_1's rmse: 0.15544
[456]	training's rmse: 0.148094	vali

[585]	training's rmse: 0.145518	valid_1's rmse: 0.153729
[586]	training's rmse: 0.145482	valid_1's rmse: 0.153698
[587]	training's rmse: 0.145459	valid_1's rmse: 0.153678
[588]	training's rmse: 0.145452	valid_1's rmse: 0.153674
[589]	training's rmse: 0.14544	valid_1's rmse: 0.153663
[590]	training's rmse: 0.145428	valid_1's rmse: 0.153656
[591]	training's rmse: 0.145404	valid_1's rmse: 0.153655
[592]	training's rmse: 0.145392	valid_1's rmse: 0.153651
[593]	training's rmse: 0.145365	valid_1's rmse: 0.153632
[594]	training's rmse: 0.145359	valid_1's rmse: 0.153628
[595]	training's rmse: 0.145346	valid_1's rmse: 0.153625
[596]	training's rmse: 0.145338	valid_1's rmse: 0.153621
[597]	training's rmse: 0.145315	valid_1's rmse: 0.153621
[598]	training's rmse: 0.145279	valid_1's rmse: 0.153594
[599]	training's rmse: 0.14526	valid_1's rmse: 0.153582
[600]	training's rmse: 0.145244	valid_1's rmse: 0.153574
[601]	training's rmse: 0.145242	valid_1's rmse: 0.153573
[602]	training's rmse: 0.145228	v

...

[1449]	training's rmse: 0.136422	valid_1's rmse: 0.149473
[1450]	training's rmse: 0.136419	valid_1's rmse: 0.149473
[1451]	training's rmse: 0.136412	valid_1's rmse: 0.149475
[1452]	training's rmse: 0.136406	valid_1's rmse: 0.149471
[1453]	training's rmse: 0.136399	valid_1's rmse: 0.149468
[1454]	training's rmse: 0.136397	valid_1's rmse: 0.149466
[1455]	training's rmse: 0.136385	valid_1's rmse: 0.14946
[1456]	training's rmse: 0.136382	valid_1's rmse: 0.149463
[1457]	training's rmse: 0.136378	valid_1's rmse: 0.149463
[1458]	training's rmse: 0.136366	valid_1's rmse: 0.149459
[1459]	training's rmse: 0.136362	valid_1's rmse: 0.14946
[1460]	training's rmse: 0.136353	valid_1's rmse: 0.149459
[1461]	training's rmse: 0.136341	valid_1's rmse: 0.149453
[1462]	training's rmse: 0.136336	valid_1's rmse: 0.149453
[1463]	training's rmse: 0.136333	valid_1's rmse: 0.149453
[1464]	training's rmse: 0.136328	valid_1's rmse: 0.149449
[1465]	training's rmse: 0.136322	valid_1's rmse: 0.149446
[1466]	training'

[1591]	training's rmse: 0.135449	valid_1's rmse: 0.149263
[1592]	training's rmse: 0.135445	valid_1's rmse: 0.149262
[1593]	training's rmse: 0.135439	valid_1's rmse: 0.149268
[1594]	training's rmse: 0.135437	valid_1's rmse: 0.14927
[1595]	training's rmse: 0.135434	valid_1's rmse: 0.149274
[1596]	training's rmse: 0.135428	valid_1's rmse: 0.149272
[1597]	training's rmse: 0.135418	valid_1's rmse: 0.149271
[1598]	training's rmse: 0.135413	valid_1's rmse: 0.149276
[1599]	training's rmse: 0.135407	valid_1's rmse: 0.149284
[1600]	training's rmse: 0.135397	valid_1's rmse: 0.149277
[1601]	training's rmse: 0.135388	valid_1's rmse: 0.14928
[1602]	training's rmse: 0.135376	valid_1's rmse: 0.149277
[1603]	training's rmse: 0.135371	valid_1's rmse: 0.149281
[1604]	training's rmse: 0.135364	valid_1's rmse: 0.149278
[1605]	training's rmse: 0.135358	valid_1's rmse: 0.149276
[1606]	training's rmse: 0.135354	valid_1's rmse: 0.149279
[1607]	training's rmse: 0.135339	valid_1's rmse: 0.149275
[1608]	training'

[1732]	training's rmse: 0.134574	valid_1's rmse: 0.149079
[1733]	training's rmse: 0.134569	valid_1's rmse: 0.149075
[1734]	training's rmse: 0.134549	valid_1's rmse: 0.149062
[1735]	training's rmse: 0.13454	valid_1's rmse: 0.149065
[1736]	training's rmse: 0.13453	valid_1's rmse: 0.149057
[1737]	training's rmse: 0.134524	valid_1's rmse: 0.149055
[1738]	training's rmse: 0.134516	valid_1's rmse: 0.149058
[1739]	training's rmse: 0.134508	valid_1's rmse: 0.149059
[1740]	training's rmse: 0.134502	valid_1's rmse: 0.149056
[1741]	training's rmse: 0.134494	valid_1's rmse: 0.149056
[1742]	training's rmse: 0.13449	valid_1's rmse: 0.149058
[1743]	training's rmse: 0.134485	valid_1's rmse: 0.149054
[1744]	training's rmse: 0.13448	valid_1's rmse: 0.149054
[1745]	training's rmse: 0.134476	valid_1's rmse: 0.149052
[1746]	training's rmse: 0.134472	valid_1's rmse: 0.149049
[1747]	training's rmse: 0.13447	valid_1's rmse: 0.14905
[1748]	training's rmse: 0.134465	valid_1's rmse: 0.149057
[1749]	training's rm

[1876]	training's rmse: 0.133761	valid_1's rmse: 0.148977
[1877]	training's rmse: 0.13375	valid_1's rmse: 0.148973
[1878]	training's rmse: 0.133747	valid_1's rmse: 0.148975
[1879]	training's rmse: 0.133746	valid_1's rmse: 0.148975
[1880]	training's rmse: 0.133744	valid_1's rmse: 0.148976
[1881]	training's rmse: 0.13374	valid_1's rmse: 0.148974
[1882]	training's rmse: 0.133727	valid_1's rmse: 0.148975
[1883]	training's rmse: 0.133719	valid_1's rmse: 0.14898
[1884]	training's rmse: 0.133719	valid_1's rmse: 0.14898
[1885]	training's rmse: 0.133714	valid_1's rmse: 0.148979
[1886]	training's rmse: 0.133705	valid_1's rmse: 0.148981
[1887]	training's rmse: 0.133698	valid_1's rmse: 0.148982
[1888]	training's rmse: 0.133687	valid_1's rmse: 0.148978
[1889]	training's rmse: 0.133683	valid_1's rmse: 0.148978
[1890]	training's rmse: 0.133676	valid_1's rmse: 0.148977
[1891]	training's rmse: 0.133666	valid_1's rmse: 0.148978
[1892]	training's rmse: 0.13366	valid_1's rmse: 0.148975
[1893]	training's r

[2020]	training's rmse: 0.132997	valid_1's rmse: 0.148933
[2021]	training's rmse: 0.13299	valid_1's rmse: 0.148934
[2022]	training's rmse: 0.132985	valid_1's rmse: 0.148934
[2023]	training's rmse: 0.132983	valid_1's rmse: 0.148936
[2024]	training's rmse: 0.132977	valid_1's rmse: 0.148942
[2025]	training's rmse: 0.132973	valid_1's rmse: 0.148948
[2026]	training's rmse: 0.132969	valid_1's rmse: 0.14895
[2027]	training's rmse: 0.132964	valid_1's rmse: 0.148953
[2028]	training's rmse: 0.132959	valid_1's rmse: 0.148955
[2029]	training's rmse: 0.132957	valid_1's rmse: 0.148956
[2030]	training's rmse: 0.132954	valid_1's rmse: 0.148955
[2031]	training's rmse: 0.132952	valid_1's rmse: 0.148957
[2032]	training's rmse: 0.132948	valid_1's rmse: 0.148953
[2033]	training's rmse: 0.132943	valid_1's rmse: 0.148954
[2034]	training's rmse: 0.132936	valid_1's rmse: 0.14895
[2035]	training's rmse: 0.132931	valid_1's rmse: 0.148951
[2036]	training's rmse: 0.132929	valid_1's rmse: 0.148951
[2037]	training's

In [ ]:
# N=1500
# The rmlse of val is: 0.074099
# The rmlse of offline is: 0.073976

#     N=2000
#     The rmlse of val is: 0.073642
# The rmlse of offline is: 0.073546

#     N=2500
#     The rmlse of val is: 0.073642
# The rmlse of offline is: 0.073545

In [58]:
get_lgb(data=data1,i=1,N=2500)

(567719, 39) (34104, 39)
[1]	training's rmse: 0.943659	valid_1's rmse: 0.956605
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 0.898686	valid_1's rmse: 0.910827
[3]	training's rmse: 0.856048	valid_1's rmse: 0.867616
[4]	training's rmse: 0.815653	valid_1's rmse: 0.826457
[5]	training's rmse: 0.777377	valid_1's rmse: 0.787464
[6]	training's rmse: 0.741162	valid_1's rmse: 0.750723
[7]	training's rmse: 0.706819	valid_1's rmse: 0.715779
[8]	training's rmse: 0.674389	valid_1's rmse: 0.682824
[9]	training's rmse: 0.643591	valid_1's rmse: 0.651472
[10]	training's rmse: 0.614412	valid_1's rmse: 0.621825
[11]	training's rmse: 0.586844	valid_1's rmse: 0.593864
[12]	training's rmse: 0.560744	valid_1's rmse: 0.567435
[13]	training's rmse: 0.53613	valid_1's rmse: 0.54239
[14]	training's rmse: 0.512832	valid_1's rmse: 0.518754
[15]	training's rmse: 0.490886	valid_1's rmse: 0.496474
[16]	training's rmse: 0.470157	valid_1's rmse: 0.475478
[17]	training's rmse: 0.450

[146]	training's rmse: 0.162319	valid_1's rmse: 0.16722
[147]	training's rmse: 0.162185	valid_1's rmse: 0.167093
[148]	training's rmse: 0.1621	valid_1's rmse: 0.167009
[149]	training's rmse: 0.161955	valid_1's rmse: 0.16688
[150]	training's rmse: 0.161819	valid_1's rmse: 0.166761
[151]	training's rmse: 0.161678	valid_1's rmse: 0.166627
[152]	training's rmse: 0.161548	valid_1's rmse: 0.166501
[153]	training's rmse: 0.161405	valid_1's rmse: 0.166359
[154]	training's rmse: 0.16123	valid_1's rmse: 0.16618
[155]	training's rmse: 0.161092	valid_1's rmse: 0.16604
[156]	training's rmse: 0.160945	valid_1's rmse: 0.165896
[157]	training's rmse: 0.160828	valid_1's rmse: 0.165779
[158]	training's rmse: 0.160771	valid_1's rmse: 0.165726
[159]	training's rmse: 0.160691	valid_1's rmse: 0.165654
[160]	training's rmse: 0.160586	valid_1's rmse: 0.165547
[161]	training's rmse: 0.160489	valid_1's rmse: 0.165471
[162]	training's rmse: 0.160396	valid_1's rmse: 0.165388
[163]	training's rmse: 0.160316	valid_

[293]	training's rmse: 0.152564	valid_1's rmse: 0.158688
[294]	training's rmse: 0.152549	valid_1's rmse: 0.158681
[295]	training's rmse: 0.152517	valid_1's rmse: 0.158652
[296]	training's rmse: 0.152465	valid_1's rmse: 0.158611
[297]	training's rmse: 0.15244	valid_1's rmse: 0.158593
[298]	training's rmse: 0.1524	valid_1's rmse: 0.15856
[299]	training's rmse: 0.152383	valid_1's rmse: 0.158549
[300]	training's rmse: 0.152356	valid_1's rmse: 0.158534
[301]	training's rmse: 0.152313	valid_1's rmse: 0.158504
[302]	training's rmse: 0.152279	valid_1's rmse: 0.158474
[303]	training's rmse: 0.152259	valid_1's rmse: 0.158457
[304]	training's rmse: 0.152211	valid_1's rmse: 0.15842
[305]	training's rmse: 0.152185	valid_1's rmse: 0.158399
[306]	training's rmse: 0.152161	valid_1's rmse: 0.158376
[307]	training's rmse: 0.152124	valid_1's rmse: 0.158349
[308]	training's rmse: 0.152099	valid_1's rmse: 0.158336
[309]	training's rmse: 0.152074	valid_1's rmse: 0.158323
[310]	training's rmse: 0.152045	vali

[438]	training's rmse: 0.148496	valid_1's rmse: 0.155684
[439]	training's rmse: 0.148455	valid_1's rmse: 0.155651
[440]	training's rmse: 0.148431	valid_1's rmse: 0.155636
[441]	training's rmse: 0.148405	valid_1's rmse: 0.155625
[442]	training's rmse: 0.148388	valid_1's rmse: 0.155617
[443]	training's rmse: 0.148362	valid_1's rmse: 0.1556
[444]	training's rmse: 0.148346	valid_1's rmse: 0.155592
[445]	training's rmse: 0.14832	valid_1's rmse: 0.155575
[446]	training's rmse: 0.148296	valid_1's rmse: 0.15557
[447]	training's rmse: 0.148262	valid_1's rmse: 0.155548
[448]	training's rmse: 0.148245	valid_1's rmse: 0.155535
[449]	training's rmse: 0.148222	valid_1's rmse: 0.155516
[450]	training's rmse: 0.148201	valid_1's rmse: 0.155502
[451]	training's rmse: 0.148165	valid_1's rmse: 0.155469
[452]	training's rmse: 0.148154	valid_1's rmse: 0.155464
[453]	training's rmse: 0.148145	valid_1's rmse: 0.155459
[454]	training's rmse: 0.148128	valid_1's rmse: 0.155448
[455]	training's rmse: 0.148118	val

[585]	training's rmse: 0.145518	valid_1's rmse: 0.153729
[586]	training's rmse: 0.145482	valid_1's rmse: 0.153698
[587]	training's rmse: 0.145459	valid_1's rmse: 0.153678
[588]	training's rmse: 0.145452	valid_1's rmse: 0.153674
[589]	training's rmse: 0.14544	valid_1's rmse: 0.153663
[590]	training's rmse: 0.145428	valid_1's rmse: 0.153656
[591]	training's rmse: 0.145404	valid_1's rmse: 0.153655
[592]	training's rmse: 0.145392	valid_1's rmse: 0.153651
[593]	training's rmse: 0.145365	valid_1's rmse: 0.153632
[594]	training's rmse: 0.145359	valid_1's rmse: 0.153628
[595]	training's rmse: 0.145346	valid_1's rmse: 0.153625
[596]	training's rmse: 0.145338	valid_1's rmse: 0.153621
[597]	training's rmse: 0.145315	valid_1's rmse: 0.153621
[598]	training's rmse: 0.145279	valid_1's rmse: 0.153594
[599]	training's rmse: 0.14526	valid_1's rmse: 0.153582
[600]	training's rmse: 0.145244	valid_1's rmse: 0.153574
[601]	training's rmse: 0.145242	valid_1's rmse: 0.153573
[602]	training's rmse: 0.145228	v

...

[1453]	training's rmse: 0.136399	valid_1's rmse: 0.149468
[1454]	training's rmse: 0.136397	valid_1's rmse: 0.149466
[1455]	training's rmse: 0.136385	valid_1's rmse: 0.14946
[1456]	training's rmse: 0.136382	valid_1's rmse: 0.149463
[1457]	training's rmse: 0.136378	valid_1's rmse: 0.149463
[1458]	training's rmse: 0.136366	valid_1's rmse: 0.149459
[1459]	training's rmse: 0.136362	valid_1's rmse: 0.14946
[1460]	training's rmse: 0.136353	valid_1's rmse: 0.149459
[1461]	training's rmse: 0.136341	valid_1's rmse: 0.149453
[1462]	training's rmse: 0.136336	valid_1's rmse: 0.149453
[1463]	training's rmse: 0.136333	valid_1's rmse: 0.149453
[1464]	training's rmse: 0.136328	valid_1's rmse: 0.149449
[1465]	training's rmse: 0.136322	valid_1's rmse: 0.149446
[1466]	training's rmse: 0.136311	valid_1's rmse: 0.149444
[1467]	training's rmse: 0.136303	valid_1's rmse: 0.149443
[1468]	training's rmse: 0.136296	valid_1's rmse: 0.149443
[1469]	training's rmse: 0.136287	valid_1's rmse: 0.149441
[1470]	training'

[1596]	training's rmse: 0.135428	valid_1's rmse: 0.149272
[1597]	training's rmse: 0.135418	valid_1's rmse: 0.149271
[1598]	training's rmse: 0.135413	valid_1's rmse: 0.149276
[1599]	training's rmse: 0.135407	valid_1's rmse: 0.149284
[1600]	training's rmse: 0.135397	valid_1's rmse: 0.149277
[1601]	training's rmse: 0.135388	valid_1's rmse: 0.14928
[1602]	training's rmse: 0.135376	valid_1's rmse: 0.149277
[1603]	training's rmse: 0.135371	valid_1's rmse: 0.149281
[1604]	training's rmse: 0.135364	valid_1's rmse: 0.149278
[1605]	training's rmse: 0.135358	valid_1's rmse: 0.149276
[1606]	training's rmse: 0.135354	valid_1's rmse: 0.149279
[1607]	training's rmse: 0.135339	valid_1's rmse: 0.149275
[1608]	training's rmse: 0.135332	valid_1's rmse: 0.149275
[1609]	training's rmse: 0.135328	valid_1's rmse: 0.149273
[1610]	training's rmse: 0.135327	valid_1's rmse: 0.149276
[1611]	training's rmse: 0.13531	valid_1's rmse: 0.149268
[1612]	training's rmse: 0.135305	valid_1's rmse: 0.149267
[1613]	training'

[1740]	training's rmse: 0.134502	valid_1's rmse: 0.149056
[1741]	training's rmse: 0.134494	valid_1's rmse: 0.149056
[1742]	training's rmse: 0.13449	valid_1's rmse: 0.149058
[1743]	training's rmse: 0.134485	valid_1's rmse: 0.149054
[1744]	training's rmse: 0.13448	valid_1's rmse: 0.149054
[1745]	training's rmse: 0.134476	valid_1's rmse: 0.149052
[1746]	training's rmse: 0.134472	valid_1's rmse: 0.149049
[1747]	training's rmse: 0.13447	valid_1's rmse: 0.14905
[1748]	training's rmse: 0.134465	valid_1's rmse: 0.149057
[1749]	training's rmse: 0.134456	valid_1's rmse: 0.149051
[1750]	training's rmse: 0.134454	valid_1's rmse: 0.149052
[1751]	training's rmse: 0.134448	valid_1's rmse: 0.149048
[1752]	training's rmse: 0.134446	valid_1's rmse: 0.149049
[1753]	training's rmse: 0.134441	valid_1's rmse: 0.149047
[1754]	training's rmse: 0.134437	valid_1's rmse: 0.14905
[1755]	training's rmse: 0.134431	valid_1's rmse: 0.149046
[1756]	training's rmse: 0.134424	valid_1's rmse: 0.149048
[1757]	training's r

[1883]	training's rmse: 0.133719	valid_1's rmse: 0.14898
[1884]	training's rmse: 0.133719	valid_1's rmse: 0.14898
[1885]	training's rmse: 0.133714	valid_1's rmse: 0.148979
[1886]	training's rmse: 0.133705	valid_1's rmse: 0.148981
[1887]	training's rmse: 0.133698	valid_1's rmse: 0.148982
[1888]	training's rmse: 0.133687	valid_1's rmse: 0.148978
[1889]	training's rmse: 0.133683	valid_1's rmse: 0.148978
[1890]	training's rmse: 0.133676	valid_1's rmse: 0.148977
[1891]	training's rmse: 0.133666	valid_1's rmse: 0.148978
[1892]	training's rmse: 0.13366	valid_1's rmse: 0.148975
[1893]	training's rmse: 0.133658	valid_1's rmse: 0.148975
[1894]	training's rmse: 0.133653	valid_1's rmse: 0.148978
[1895]	training's rmse: 0.13365	valid_1's rmse: 0.148976
[1896]	training's rmse: 0.13364	valid_1's rmse: 0.148971
[1897]	training's rmse: 0.133633	valid_1's rmse: 0.148968
[1898]	training's rmse: 0.13363	valid_1's rmse: 0.148966
[1899]	training's rmse: 0.13363	valid_1's rmse: 0.148966
[1900]	training's rms

[2029]	training's rmse: 0.132957	valid_1's rmse: 0.148956
[2030]	training's rmse: 0.132954	valid_1's rmse: 0.148955
[2031]	training's rmse: 0.132952	valid_1's rmse: 0.148957
[2032]	training's rmse: 0.132948	valid_1's rmse: 0.148953
[2033]	training's rmse: 0.132943	valid_1's rmse: 0.148954
[2034]	training's rmse: 0.132936	valid_1's rmse: 0.14895
[2035]	training's rmse: 0.132931	valid_1's rmse: 0.148951
[2036]	training's rmse: 0.132929	valid_1's rmse: 0.148951
[2037]	training's rmse: 0.132927	valid_1's rmse: 0.14895
[2038]	training's rmse: 0.132922	valid_1's rmse: 0.148953
[2039]	training's rmse: 0.132914	valid_1's rmse: 0.148958
[2040]	training's rmse: 0.13291	valid_1's rmse: 0.148958
[2041]	training's rmse: 0.132892	valid_1's rmse: 0.148944
[2042]	training's rmse: 0.132888	valid_1's rmse: 0.148945
[2043]	training's rmse: 0.132885	valid_1's rmse: 0.148946
[2044]	training's rmse: 0.132882	valid_1's rmse: 0.148946
[2045]	training's rmse: 0.132869	valid_1's rmse: 0.148944
[2046]	training's

# ======1 over=========

In [66]:
# 5
gc.collect()
data5=pd.read_csv('../data/data_5.csv')
get_org_lgb(data=data5,N=2000)

(556288, 39) (34044, 39)
[1]	training's rmse: 1.36321	valid_1's rmse: 1.38596
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 1.29821	valid_1's rmse: 1.31986
[3]	training's rmse: 1.23661	valid_1's rmse: 1.25712
[4]	training's rmse: 1.17846	valid_1's rmse: 1.1979
[5]	training's rmse: 1.12312	valid_1's rmse: 1.14158
[6]	training's rmse: 1.07095	valid_1's rmse: 1.0885
[7]	training's rmse: 1.02149	valid_1's rmse: 1.03815
[8]	training's rmse: 0.974765	valid_1's rmse: 0.990679
[9]	training's rmse: 0.930218	valid_1's rmse: 0.945318
[10]	training's rmse: 0.887989	valid_1's rmse: 0.902343
[11]	training's rmse: 0.848096	valid_1's rmse: 0.861772
[12]	training's rmse: 0.810322	valid_1's rmse: 0.823351
[13]	training's rmse: 0.77468	valid_1's rmse: 0.787095
[14]	training's rmse: 0.741035	valid_1's rmse: 0.752826
[15]	training's rmse: 0.709279	valid_1's rmse: 0.720503
[16]	training's rmse: 0.679225	valid_1's rmse: 0.689874
[17]	training's rmse: 0.650957	valid_1's r

[148]	training's rmse: 0.216683	valid_1's rmse: 0.221722
[149]	training's rmse: 0.216458	valid_1's rmse: 0.221509
[150]	training's rmse: 0.216334	valid_1's rmse: 0.221382
[151]	training's rmse: 0.216215	valid_1's rmse: 0.221298
[152]	training's rmse: 0.216052	valid_1's rmse: 0.221151
[153]	training's rmse: 0.215939	valid_1's rmse: 0.221047
[154]	training's rmse: 0.215885	valid_1's rmse: 0.221003
[155]	training's rmse: 0.215663	valid_1's rmse: 0.220809
[156]	training's rmse: 0.21555	valid_1's rmse: 0.220692
[157]	training's rmse: 0.215412	valid_1's rmse: 0.220563
[158]	training's rmse: 0.215307	valid_1's rmse: 0.220493
[159]	training's rmse: 0.215134	valid_1's rmse: 0.220342
[160]	training's rmse: 0.215035	valid_1's rmse: 0.220242
[161]	training's rmse: 0.214931	valid_1's rmse: 0.220151
[162]	training's rmse: 0.214832	valid_1's rmse: 0.220048
[163]	training's rmse: 0.214642	valid_1's rmse: 0.219916
[164]	training's rmse: 0.214515	valid_1's rmse: 0.219823
[165]	training's rmse: 0.214392	

[294]	training's rmse: 0.20301	valid_1's rmse: 0.209856
[295]	training's rmse: 0.202982	valid_1's rmse: 0.209831
[296]	training's rmse: 0.202902	valid_1's rmse: 0.209807
[297]	training's rmse: 0.202841	valid_1's rmse: 0.209754
[298]	training's rmse: 0.20277	valid_1's rmse: 0.209702
[299]	training's rmse: 0.202635	valid_1's rmse: 0.20958
[300]	training's rmse: 0.202579	valid_1's rmse: 0.209533
[301]	training's rmse: 0.202531	valid_1's rmse: 0.209494
[302]	training's rmse: 0.202442	valid_1's rmse: 0.209458
[303]	training's rmse: 0.202359	valid_1's rmse: 0.209384
[304]	training's rmse: 0.202298	valid_1's rmse: 0.209337
[305]	training's rmse: 0.202254	valid_1's rmse: 0.209315
[306]	training's rmse: 0.202191	valid_1's rmse: 0.209281
[307]	training's rmse: 0.202142	valid_1's rmse: 0.209247
[308]	training's rmse: 0.202087	valid_1's rmse: 0.209195
[309]	training's rmse: 0.202031	valid_1's rmse: 0.209149
[310]	training's rmse: 0.201994	valid_1's rmse: 0.209124
[311]	training's rmse: 0.201954	va

[442]	training's rmse: 0.196297	valid_1's rmse: 0.204807
[443]	training's rmse: 0.196258	valid_1's rmse: 0.204764
[444]	training's rmse: 0.196229	valid_1's rmse: 0.204756
[445]	training's rmse: 0.196185	valid_1's rmse: 0.204712
[446]	training's rmse: 0.196158	valid_1's rmse: 0.20469
[447]	training's rmse: 0.196118	valid_1's rmse: 0.204658
[448]	training's rmse: 0.196092	valid_1's rmse: 0.204641
[449]	training's rmse: 0.196074	valid_1's rmse: 0.204621
[450]	training's rmse: 0.196065	valid_1's rmse: 0.204612
[451]	training's rmse: 0.196049	valid_1's rmse: 0.204599
[452]	training's rmse: 0.196	valid_1's rmse: 0.204555
[453]	training's rmse: 0.195974	valid_1's rmse: 0.204531
[454]	training's rmse: 0.195942	valid_1's rmse: 0.204514
[455]	training's rmse: 0.1959	valid_1's rmse: 0.204511
[456]	training's rmse: 0.195883	valid_1's rmse: 0.204499
[457]	training's rmse: 0.195863	valid_1's rmse: 0.204483
[458]	training's rmse: 0.195846	valid_1's rmse: 0.204473
[459]	training's rmse: 0.195806	valid

[588]	training's rmse: 0.192052	valid_1's rmse: 0.202234
[589]	training's rmse: 0.192017	valid_1's rmse: 0.202216
[590]	training's rmse: 0.191989	valid_1's rmse: 0.202198
[591]	training's rmse: 0.191976	valid_1's rmse: 0.202187
[592]	training's rmse: 0.191915	valid_1's rmse: 0.202164
[593]	training's rmse: 0.191878	valid_1's rmse: 0.202131
[594]	training's rmse: 0.191847	valid_1's rmse: 0.202111
[595]	training's rmse: 0.191823	valid_1's rmse: 0.202119
[596]	training's rmse: 0.191787	valid_1's rmse: 0.202098
[597]	training's rmse: 0.191776	valid_1's rmse: 0.202088
[598]	training's rmse: 0.191732	valid_1's rmse: 0.202085
[599]	training's rmse: 0.191706	valid_1's rmse: 0.202067
[600]	training's rmse: 0.191696	valid_1's rmse: 0.202061
[601]	training's rmse: 0.191686	valid_1's rmse: 0.202056
[602]	training's rmse: 0.191665	valid_1's rmse: 0.202063
[603]	training's rmse: 0.19163	valid_1's rmse: 0.202045
[604]	training's rmse: 0.191615	valid_1's rmse: 0.202033
[605]	training's rmse: 0.191562	

...

[882]	training's rmse: 0.186364	valid_1's rmse: 0.19971
[883]	training's rmse: 0.186338	valid_1's rmse: 0.199693
[884]	training's rmse: 0.186308	valid_1's rmse: 0.199705
[885]	training's rmse: 0.186303	valid_1's rmse: 0.199703
[886]	training's rmse: 0.186285	valid_1's rmse: 0.199697
[887]	training's rmse: 0.186274	valid_1's rmse: 0.199692
[888]	training's rmse: 0.186262	valid_1's rmse: 0.19969
[889]	training's rmse: 0.186255	valid_1's rmse: 0.199695
[890]	training's rmse: 0.186246	valid_1's rmse: 0.199695
[891]	training's rmse: 0.186233	valid_1's rmse: 0.199695
[892]	training's rmse: 0.18623	valid_1's rmse: 0.199693
[893]	training's rmse: 0.186226	valid_1's rmse: 0.19969
[894]	training's rmse: 0.186222	valid_1's rmse: 0.199687
[895]	training's rmse: 0.186189	valid_1's rmse: 0.19966
[896]	training's rmse: 0.186185	valid_1's rmse: 0.199658
[897]	training's rmse: 0.186173	valid_1's rmse: 0.199647
[898]	training's rmse: 0.186166	valid_1's rmse: 0.199639
[899]	training's rmse: 0.186147	vali

[1026]	training's rmse: 0.184382	valid_1's rmse: 0.199062
[1027]	training's rmse: 0.184367	valid_1's rmse: 0.199056
[1028]	training's rmse: 0.184353	valid_1's rmse: 0.199053
[1029]	training's rmse: 0.184332	valid_1's rmse: 0.199062
[1030]	training's rmse: 0.184316	valid_1's rmse: 0.199059
[1031]	training's rmse: 0.184301	valid_1's rmse: 0.199051
[1032]	training's rmse: 0.184281	valid_1's rmse: 0.19905
[1033]	training's rmse: 0.18427	valid_1's rmse: 0.199042
[1034]	training's rmse: 0.184251	valid_1's rmse: 0.199028
[1035]	training's rmse: 0.184233	valid_1's rmse: 0.199034
[1036]	training's rmse: 0.18421	valid_1's rmse: 0.199038
[1037]	training's rmse: 0.184201	valid_1's rmse: 0.199034
[1038]	training's rmse: 0.184178	valid_1's rmse: 0.199049
[1039]	training's rmse: 0.184168	valid_1's rmse: 0.199042
[1040]	training's rmse: 0.184157	valid_1's rmse: 0.19904
[1041]	training's rmse: 0.18414	valid_1's rmse: 0.19904
[1042]	training's rmse: 0.184119	valid_1's rmse: 0.199048
[1043]	training's rm

[1169]	training's rmse: 0.182612	valid_1's rmse: 0.1987
[1170]	training's rmse: 0.182598	valid_1's rmse: 0.198689
[1171]	training's rmse: 0.182589	valid_1's rmse: 0.19869
[1172]	training's rmse: 0.18256	valid_1's rmse: 0.198693
[1173]	training's rmse: 0.182554	valid_1's rmse: 0.198689
[1174]	training's rmse: 0.182551	valid_1's rmse: 0.198689
[1175]	training's rmse: 0.182539	valid_1's rmse: 0.198689
[1176]	training's rmse: 0.182537	valid_1's rmse: 0.198689
[1177]	training's rmse: 0.182522	valid_1's rmse: 0.198686
[1178]	training's rmse: 0.182509	valid_1's rmse: 0.198683
[1179]	training's rmse: 0.182498	valid_1's rmse: 0.198678
[1180]	training's rmse: 0.182484	valid_1's rmse: 0.198684
[1181]	training's rmse: 0.182476	valid_1's rmse: 0.198715
[1182]	training's rmse: 0.182469	valid_1's rmse: 0.198714
[1183]	training's rmse: 0.182464	valid_1's rmse: 0.19871
[1184]	training's rmse: 0.182451	valid_1's rmse: 0.198704
[1185]	training's rmse: 0.182434	valid_1's rmse: 0.198702
[1186]	training's r

[1313]	training's rmse: 0.181138	valid_1's rmse: 0.19863
[1314]	training's rmse: 0.181133	valid_1's rmse: 0.198628
[1315]	training's rmse: 0.181127	valid_1's rmse: 0.198626
[1316]	training's rmse: 0.181124	valid_1's rmse: 0.198624
[1317]	training's rmse: 0.18112	valid_1's rmse: 0.19862
[1318]	training's rmse: 0.181117	valid_1's rmse: 0.198619
[1319]	training's rmse: 0.1811	valid_1's rmse: 0.198611
[1320]	training's rmse: 0.181077	valid_1's rmse: 0.198621
[1321]	training's rmse: 0.18107	valid_1's rmse: 0.198616
[1322]	training's rmse: 0.181063	valid_1's rmse: 0.198623
[1323]	training's rmse: 0.181054	valid_1's rmse: 0.198622
[1324]	training's rmse: 0.18105	valid_1's rmse: 0.198621
[1325]	training's rmse: 0.181047	valid_1's rmse: 0.198623
[1326]	training's rmse: 0.181039	valid_1's rmse: 0.198621
[1327]	training's rmse: 0.181027	valid_1's rmse: 0.198609
[1328]	training's rmse: 0.181021	valid_1's rmse: 0.198605
[1329]	training's rmse: 0.18101	valid_1's rmse: 0.1986
[1330]	training's rmse: 

[1457]	training's rmse: 0.179924	valid_1's rmse: 0.19869
[1458]	training's rmse: 0.179904	valid_1's rmse: 0.198676
[1459]	training's rmse: 0.179894	valid_1's rmse: 0.198681
[1460]	training's rmse: 0.179889	valid_1's rmse: 0.198683
[1461]	training's rmse: 0.179886	valid_1's rmse: 0.198682
[1462]	training's rmse: 0.179874	valid_1's rmse: 0.198684
[1463]	training's rmse: 0.179865	valid_1's rmse: 0.198701
[1464]	training's rmse: 0.179845	valid_1's rmse: 0.1987
[1465]	training's rmse: 0.179841	valid_1's rmse: 0.198702
[1466]	training's rmse: 0.179834	valid_1's rmse: 0.198701
[1467]	training's rmse: 0.179831	valid_1's rmse: 0.198699
[1468]	training's rmse: 0.179824	valid_1's rmse: 0.198708
[1469]	training's rmse: 0.179815	valid_1's rmse: 0.198704
[1470]	training's rmse: 0.179813	valid_1's rmse: 0.198702
Early stopping, best iteration is:
[1370]	training's rmse: 0.180717	valid_1's rmse: 0.198557
The rmlse of val is: 0.083718
The rmlse of offline is: 0.084204


In [67]:
get_lgb(data=data5,i=5,N=2000)

(556288, 39) (34044, 39)
[1]	training's rmse: 1.36321	valid_1's rmse: 1.38596
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 1.29821	valid_1's rmse: 1.31986
[3]	training's rmse: 1.23661	valid_1's rmse: 1.25712
[4]	training's rmse: 1.17846	valid_1's rmse: 1.1979
[5]	training's rmse: 1.12312	valid_1's rmse: 1.14158
[6]	training's rmse: 1.07095	valid_1's rmse: 1.0885
[7]	training's rmse: 1.02149	valid_1's rmse: 1.03815
[8]	training's rmse: 0.974765	valid_1's rmse: 0.990679
[9]	training's rmse: 0.930218	valid_1's rmse: 0.945318
[10]	training's rmse: 0.887989	valid_1's rmse: 0.902343
[11]	training's rmse: 0.848096	valid_1's rmse: 0.861772
[12]	training's rmse: 0.810322	valid_1's rmse: 0.823351
[13]	training's rmse: 0.77468	valid_1's rmse: 0.787095
[14]	training's rmse: 0.741035	valid_1's rmse: 0.752826
[15]	training's rmse: 0.709279	valid_1's rmse: 0.720503
[16]	training's rmse: 0.679225	valid_1's rmse: 0.689874
[17]	training's rmse: 0.650957	valid_1's r

[149]	training's rmse: 0.216458	valid_1's rmse: 0.221509
[150]	training's rmse: 0.216334	valid_1's rmse: 0.221382
[151]	training's rmse: 0.216215	valid_1's rmse: 0.221298
[152]	training's rmse: 0.216052	valid_1's rmse: 0.221151
[153]	training's rmse: 0.215939	valid_1's rmse: 0.221047
[154]	training's rmse: 0.215885	valid_1's rmse: 0.221003
[155]	training's rmse: 0.215663	valid_1's rmse: 0.220809
[156]	training's rmse: 0.21555	valid_1's rmse: 0.220692
[157]	training's rmse: 0.215412	valid_1's rmse: 0.220563
[158]	training's rmse: 0.215307	valid_1's rmse: 0.220493
[159]	training's rmse: 0.215134	valid_1's rmse: 0.220342
[160]	training's rmse: 0.215035	valid_1's rmse: 0.220242
[161]	training's rmse: 0.214931	valid_1's rmse: 0.220151
[162]	training's rmse: 0.214832	valid_1's rmse: 0.220048
[163]	training's rmse: 0.214642	valid_1's rmse: 0.219916
[164]	training's rmse: 0.214515	valid_1's rmse: 0.219823
[165]	training's rmse: 0.214392	valid_1's rmse: 0.219709
[166]	training's rmse: 0.214286	

[297]	training's rmse: 0.202841	valid_1's rmse: 0.209754
[298]	training's rmse: 0.20277	valid_1's rmse: 0.209702
[299]	training's rmse: 0.202635	valid_1's rmse: 0.20958
[300]	training's rmse: 0.202579	valid_1's rmse: 0.209533
[301]	training's rmse: 0.202531	valid_1's rmse: 0.209494
[302]	training's rmse: 0.202442	valid_1's rmse: 0.209458
[303]	training's rmse: 0.202359	valid_1's rmse: 0.209384
[304]	training's rmse: 0.202298	valid_1's rmse: 0.209337
[305]	training's rmse: 0.202254	valid_1's rmse: 0.209315
[306]	training's rmse: 0.202191	valid_1's rmse: 0.209281
[307]	training's rmse: 0.202142	valid_1's rmse: 0.209247
[308]	training's rmse: 0.202087	valid_1's rmse: 0.209195
[309]	training's rmse: 0.202031	valid_1's rmse: 0.209149
[310]	training's rmse: 0.201994	valid_1's rmse: 0.209124
[311]	training's rmse: 0.201954	valid_1's rmse: 0.20908
[312]	training's rmse: 0.201914	valid_1's rmse: 0.20905
[313]	training's rmse: 0.201867	valid_1's rmse: 0.20901
[314]	training's rmse: 0.201835	vali

[443]	training's rmse: 0.196258	valid_1's rmse: 0.204764
[444]	training's rmse: 0.196229	valid_1's rmse: 0.204756
[445]	training's rmse: 0.196185	valid_1's rmse: 0.204712
[446]	training's rmse: 0.196158	valid_1's rmse: 0.20469
[447]	training's rmse: 0.196118	valid_1's rmse: 0.204658
[448]	training's rmse: 0.196092	valid_1's rmse: 0.204641
[449]	training's rmse: 0.196074	valid_1's rmse: 0.204621
[450]	training's rmse: 0.196065	valid_1's rmse: 0.204612
[451]	training's rmse: 0.196049	valid_1's rmse: 0.204599
[452]	training's rmse: 0.196	valid_1's rmse: 0.204555
[453]	training's rmse: 0.195974	valid_1's rmse: 0.204531
[454]	training's rmse: 0.195942	valid_1's rmse: 0.204514
[455]	training's rmse: 0.1959	valid_1's rmse: 0.204511
[456]	training's rmse: 0.195883	valid_1's rmse: 0.204499
[457]	training's rmse: 0.195863	valid_1's rmse: 0.204483
[458]	training's rmse: 0.195846	valid_1's rmse: 0.204473
[459]	training's rmse: 0.195806	valid_1's rmse: 0.204441
[460]	training's rmse: 0.195743	valid

[589]	training's rmse: 0.192017	valid_1's rmse: 0.202216
[590]	training's rmse: 0.191989	valid_1's rmse: 0.202198
[591]	training's rmse: 0.191976	valid_1's rmse: 0.202187
[592]	training's rmse: 0.191915	valid_1's rmse: 0.202164
[593]	training's rmse: 0.191878	valid_1's rmse: 0.202131
[594]	training's rmse: 0.191847	valid_1's rmse: 0.202111
[595]	training's rmse: 0.191823	valid_1's rmse: 0.202119
[596]	training's rmse: 0.191787	valid_1's rmse: 0.202098
[597]	training's rmse: 0.191776	valid_1's rmse: 0.202088
[598]	training's rmse: 0.191732	valid_1's rmse: 0.202085
[599]	training's rmse: 0.191706	valid_1's rmse: 0.202067
[600]	training's rmse: 0.191696	valid_1's rmse: 0.202061
[601]	training's rmse: 0.191686	valid_1's rmse: 0.202056
[602]	training's rmse: 0.191665	valid_1's rmse: 0.202063
[603]	training's rmse: 0.19163	valid_1's rmse: 0.202045
[604]	training's rmse: 0.191615	valid_1's rmse: 0.202033
[605]	training's rmse: 0.191562	valid_1's rmse: 0.201979
[606]	training's rmse: 0.191516	

...

[880]	training's rmse: 0.186424	valid_1's rmse: 0.19974
[881]	training's rmse: 0.186396	valid_1's rmse: 0.199733
[882]	training's rmse: 0.186364	valid_1's rmse: 0.19971
[883]	training's rmse: 0.186338	valid_1's rmse: 0.199693
[884]	training's rmse: 0.186308	valid_1's rmse: 0.199705
[885]	training's rmse: 0.186303	valid_1's rmse: 0.199703
[886]	training's rmse: 0.186285	valid_1's rmse: 0.199697
[887]	training's rmse: 0.186274	valid_1's rmse: 0.199692
[888]	training's rmse: 0.186262	valid_1's rmse: 0.19969
[889]	training's rmse: 0.186255	valid_1's rmse: 0.199695
[890]	training's rmse: 0.186246	valid_1's rmse: 0.199695
[891]	training's rmse: 0.186233	valid_1's rmse: 0.199695
[892]	training's rmse: 0.18623	valid_1's rmse: 0.199693
[893]	training's rmse: 0.186226	valid_1's rmse: 0.19969
[894]	training's rmse: 0.186222	valid_1's rmse: 0.199687
[895]	training's rmse: 0.186189	valid_1's rmse: 0.19966
[896]	training's rmse: 0.186185	valid_1's rmse: 0.199658
[897]	training's rmse: 0.186173	valid

[1026]	training's rmse: 0.184382	valid_1's rmse: 0.199062
[1027]	training's rmse: 0.184367	valid_1's rmse: 0.199056
[1028]	training's rmse: 0.184353	valid_1's rmse: 0.199053
[1029]	training's rmse: 0.184332	valid_1's rmse: 0.199062
[1030]	training's rmse: 0.184316	valid_1's rmse: 0.199059
[1031]	training's rmse: 0.184301	valid_1's rmse: 0.199051
[1032]	training's rmse: 0.184281	valid_1's rmse: 0.19905
[1033]	training's rmse: 0.18427	valid_1's rmse: 0.199042
[1034]	training's rmse: 0.184251	valid_1's rmse: 0.199028
[1035]	training's rmse: 0.184233	valid_1's rmse: 0.199034
[1036]	training's rmse: 0.18421	valid_1's rmse: 0.199038
[1037]	training's rmse: 0.184201	valid_1's rmse: 0.199034
[1038]	training's rmse: 0.184178	valid_1's rmse: 0.199049
[1039]	training's rmse: 0.184168	valid_1's rmse: 0.199042
[1040]	training's rmse: 0.184157	valid_1's rmse: 0.19904
[1041]	training's rmse: 0.18414	valid_1's rmse: 0.19904
[1042]	training's rmse: 0.184119	valid_1's rmse: 0.199048
[1043]	training's rm

[1170]	training's rmse: 0.182598	valid_1's rmse: 0.198689
[1171]	training's rmse: 0.182589	valid_1's rmse: 0.19869
[1172]	training's rmse: 0.18256	valid_1's rmse: 0.198693
[1173]	training's rmse: 0.182554	valid_1's rmse: 0.198689
[1174]	training's rmse: 0.182551	valid_1's rmse: 0.198689
[1175]	training's rmse: 0.182539	valid_1's rmse: 0.198689
[1176]	training's rmse: 0.182537	valid_1's rmse: 0.198689
[1177]	training's rmse: 0.182522	valid_1's rmse: 0.198686
[1178]	training's rmse: 0.182509	valid_1's rmse: 0.198683
[1179]	training's rmse: 0.182498	valid_1's rmse: 0.198678
[1180]	training's rmse: 0.182484	valid_1's rmse: 0.198684
[1181]	training's rmse: 0.182476	valid_1's rmse: 0.198715
[1182]	training's rmse: 0.182469	valid_1's rmse: 0.198714
[1183]	training's rmse: 0.182464	valid_1's rmse: 0.19871
[1184]	training's rmse: 0.182451	valid_1's rmse: 0.198704
[1185]	training's rmse: 0.182434	valid_1's rmse: 0.198702
[1186]	training's rmse: 0.182421	valid_1's rmse: 0.198701
[1187]	training's

[1313]	training's rmse: 0.181138	valid_1's rmse: 0.19863
[1314]	training's rmse: 0.181133	valid_1's rmse: 0.198628
[1315]	training's rmse: 0.181127	valid_1's rmse: 0.198626
[1316]	training's rmse: 0.181124	valid_1's rmse: 0.198624
[1317]	training's rmse: 0.18112	valid_1's rmse: 0.19862
[1318]	training's rmse: 0.181117	valid_1's rmse: 0.198619
[1319]	training's rmse: 0.1811	valid_1's rmse: 0.198611
[1320]	training's rmse: 0.181077	valid_1's rmse: 0.198621
[1321]	training's rmse: 0.18107	valid_1's rmse: 0.198616
[1322]	training's rmse: 0.181063	valid_1's rmse: 0.198623
[1323]	training's rmse: 0.181054	valid_1's rmse: 0.198622
[1324]	training's rmse: 0.18105	valid_1's rmse: 0.198621
[1325]	training's rmse: 0.181047	valid_1's rmse: 0.198623
[1326]	training's rmse: 0.181039	valid_1's rmse: 0.198621
[1327]	training's rmse: 0.181027	valid_1's rmse: 0.198609
[1328]	training's rmse: 0.181021	valid_1's rmse: 0.198605
[1329]	training's rmse: 0.18101	valid_1's rmse: 0.1986
[1330]	training's rmse: 

[1457]	training's rmse: 0.179924	valid_1's rmse: 0.19869
[1458]	training's rmse: 0.179904	valid_1's rmse: 0.198676
[1459]	training's rmse: 0.179894	valid_1's rmse: 0.198681
[1460]	training's rmse: 0.179889	valid_1's rmse: 0.198683
[1461]	training's rmse: 0.179886	valid_1's rmse: 0.198682
[1462]	training's rmse: 0.179874	valid_1's rmse: 0.198684
[1463]	training's rmse: 0.179865	valid_1's rmse: 0.198701
[1464]	training's rmse: 0.179845	valid_1's rmse: 0.1987
[1465]	training's rmse: 0.179841	valid_1's rmse: 0.198702
[1466]	training's rmse: 0.179834	valid_1's rmse: 0.198701
[1467]	training's rmse: 0.179831	valid_1's rmse: 0.198699
[1468]	training's rmse: 0.179824	valid_1's rmse: 0.198708
[1469]	training's rmse: 0.179815	valid_1's rmse: 0.198704
[1470]	training's rmse: 0.179813	valid_1's rmse: 0.198702
Early stopping, best iteration is:
[1370]	training's rmse: 0.180717	valid_1's rmse: 0.198557


In [68]:
# #每个城市对应的数据集
# 0    273015
# 2    256286

# 1    567719
# 5    556288

# 3    405264
# 4    386266

# 6     26437